In [1]:
from qick import *

# Main program class
from qick.asm_v2 import AveragerProgramV2

# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D
from qick.asm_v2 import QickProgramV2, AcquireProgramV2

# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger
import time

import sys
np.set_printoptions(threshold=sys.maxsize)

soc = QickSoc('qick_216_test.bit')
#soccfg = soc
#soc.tproc.info()


resetting clocks: 245.76 491.52


In [2]:
print(soc)
print(soc.tproc)



QICK configuration:

	Board: ZCU216

	Software version: 0.2.234
	Firmware timestamp: Thu Mar  7 14:30:10 2024

	Global clocks (MHz): tProcessor 500.002, RF reference 245.760

	2 signal generator channels:
	0:	axis_sg_int4_v1 - envelope memory 4096 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 16-bit DDS, range=1228.800 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (8.333 us)
		fs=7864.320 MHz, fabric=491.520 MHz, 32-bit DDS, range=7864.320 MHz
		DAC tile 3, blk 0 is 0_231, on JHC3

	2 readout channels:
	0:	axis_readout_v2 - controlled by PYNQ
		fs=2457.600 MHz, fabric=307.200 MHz, 32-bit DDS, range=2457.600 MHz
		maxlen 16384 accumulated, 1024 decimated (3.333 us)
		triggered by dport 0, pin 0, feedback to tProc input 0
		ADC tile 2, blk 0 is 0_226, on JHC7
	1:	axis_readout_v2 - controlled by PYNQ
		fs=2457.600 MHz, fabric=307.200 MHz, 32-bit DDS, range=2457.600 MHz
		maxlen 16384 accumulated, 1024 decimated (3.333 us)
		

In [3]:
## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger
import sys
import time

np.set_printoptions(threshold=sys.maxsize)

wr_buff = []
Data_qty = 1000


asm = """
REG_WR s_cfg imm #16

INIT:
   REG_WR s12  op -op(s12 + #1)
   REG_WR s13  imm #-1 -if(F)
   REG_WR s13  imm #2  -if(NF)
   JUMP INIT
JUMP HERE
"""

print('Loading Program Memory:')

p_txt, p_bin  = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)

print('Loading Data Memory:')
for i in range(Data_qty):
    wr_buff.append( [0, 0,0,0,0,0,0,0] )
soc.tproc.load_mem(2, wr_buff, 0)
#rd_buff = soc.tproc.read_mem(2, 0, Data_qty)
#print(rd_buff)

soc.tproc.read_sel  = 1


print('Start Core')
soc.tproc.start()
soc.tproc.clr_axi_flg()
print(soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2)
soc.tproc.set_axi_flg()
print(soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2)
soc.tproc.clr_axi_flg()
print(soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2)
soc.tproc.set_axi_flg()
print(soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2)
soc.tproc.clr_axi_flg()
print(soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2)
soc.tproc.set_axi_flg()
print(soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2)

#soc.tproc.get_status()
print('Stop Core')
soc.tproc.stop()

src_list = ['TPROC_W_DT','CORE0_W_DT','RFU','DIV (Q-R)','ARITH(L-H)','TNET','PERIPH','PORT','RAND','RFU','RFU','RFU','RFU','RFU','RFU','RFU',]
for ind in range(16):
    soc.tproc.read_sel  = ind
    print("{:10} ( READ_SEL={:2} ) = {:10}-{:10}".format(src_list[ind],soc.tproc.read_sel, soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2) )






Loading Program Memory:
Loading Data Memory:
Start Core
22409 2
97686 4294967295
177718 2
255883 4294967295
334180 2
413501 4294967295
Stop Core
TPROC_W_DT ( READ_SEL= 0 ) =          0-         0
CORE0_W_DT ( READ_SEL= 1 ) =     538341-4294967295
RFU        ( READ_SEL= 2 ) =          0-         0
DIV (Q-R)  ( READ_SEL= 3 ) = 4294967295-         0
ARITH(L-H) ( READ_SEL= 4 ) =          0-         0
TNET       ( READ_SEL= 5 ) =          0-         0
PERIPH     ( READ_SEL= 6 ) =          0-         0
PORT       ( READ_SEL= 7 ) =          0-         0
RAND       ( READ_SEL= 8 ) =          0-         0
RFU        ( READ_SEL= 9 ) =          0-         0
RFU        ( READ_SEL=10 ) =          0-         0
RFU        ( READ_SEL=11 ) =          0-         0
RFU        ( READ_SEL=12 ) =          0-         0
RFU        ( READ_SEL=13 ) =          0-         0
RFU        ( READ_SEL=14 ) =          0-         0
RFU        ( READ_SEL=15 ) =          0-         0


In [5]:
## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger
import sys
import time

np.set_printoptions(threshold=sys.maxsize)


Data_qty = 1000
Test_time = np.zeros(Test_qty, dtype=np.int32)
Test_val  = np.zeros(Test_qty, dtype=np.int32)


asm = """
REG_WR s_cfg imm #16
REG_WR s_cfg imm cfg_flg_axi

INIT:
   REG_WR s12  op -op(s12 + #1)
   REG_WR s13  imm #-1 -if(F)
   REG_WR s13  imm #2  -if(NF)
   JUMP INIT

"""

print('Loading Program Memory:')
p_txt, p_bin  = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)

print('Loading Data Memory:')
wr_buff = []
for i in range(Data_qty):
    wr_buff.append( [0, 0,0,0,0,0,0,0] )
soc.tproc.load_mem(2, wr_buff, 0)


Test_qty = 10
print('Start Core')
soc.tproc.start()
for i in range(0,Test_qty,2):
    soc.tproc.set_axi_flg()
    Test_time[i], Test_val[i] = soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2
    soc.tproc.clr_axi_flg()
    Test_time[i+1], Test_val[i+1] = soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2

soc.tproc.stop()

print(Test_time)
print(Test_val)





Loading Program Memory:
Loading Data Memory:
Start Core


IndexError: list assignment index out of range

In [ ]:
a, b = soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2
print(a)